# 00 - Extraccion de datos mediante API de Mercado Libre

## Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import json
import requests

from datetime import datetime, timedelta, timezone, date
pd.options.display.max_columns=None
from pyprojroot import here


import matplotlib.pyplot as plt
import os
import seaborn as sns


In [2]:
path_save = here() / "data" 

In [3]:
here()

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis')

In [4]:
import sys
sys.path.append(here())


In [5]:
here()

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis')

In [6]:
sys.path.insert(0, 'c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis/library')
import sys

from extraction import MeliAPI

## Armado del Dataset

Para armar el dataset descargaremos para Mercado Libre Argentina laptops publicadas. La categoría se llama: MLA1652.
La API tiene una restricción que no permite utilizar un offset mayor a 4000. Esto nos permite descargar menos de 4000 publicaciones por loop.
Es por ello que se han armado ciertos grupos para descargar la información por grupo. Esto nos permite tambien etiquetar a las publicaciones con este atributo, 
elemento que no se obtiene descargando la información de la publicación.

<img src="..\Grupos.png" alt="Drawing" style="width: 600px;"/>


Para armar estos grupos necesitamos filtar desde el url mediante los siguientes nombres:  

* installments:  
    + yes  
    + no_interest  
* power_seller:  
    + yes  
    + no  
* shipping_cost:  
    + get_available_filtersfree  
            

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/


In [9]:
meli_api = MeliAPI()

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/
<Response [200]>


### Grupo 1

In [10]:
filters = {"installments":"yes",
           "display_size":'(*-14.1")',
          "shipping_cost":"free"}


In [11]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/
<Response [200]>
Encontramos 4197 resultados para nuestra consulta


In [7]:
item_list = meli_api.create_item_list(filters = filters)

In [8]:
data = meli_api.get_df_list(item_list)
data["installment"] = "yes"
data["display_size"] = "hasta 14.1" 
data["shipping_cost"] = "free"

### Grupo 2

In [9]:
filters = {"installments":"yes",
           "display_size":'[14.1"-17")',
                "shipping_cost":"free"}


In [10]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 4042 resultados para nuestra consulta


In [11]:
item_list = meli_api.create_item_list(filters = filters)

El offst es 4000 se terminará la ejecución.


In [22]:
data2 = meli_api.get_df_list(item_list)
data2["installment"] = "yes"
data2["display_size"] = "entre 14.1 y 16.9"
data2["shipping_cost"] = "free"

: 

### Grupo 3

In [13]:
filters = {"installments":"no_interest",
           "display_size":'(*-14.1")',
          "shipping_cost":"free"}


In [14]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 1254 resultados para nuestra consulta


In [15]:
item_list = meli_api.create_item_list(filters = filters)

In [16]:
data3 = meli_api.get_df_list(item_list)
data3["installment"] = "no_interest"
data3["display_size"] = "hasta 14.1" 
data3["shipping_cost"] = "free"

### Grupo 4

In [17]:
filters = {"installments":"no_interest",
           "display_size":'[14.1"-17")',
          "shipping_cost":"free"}


In [18]:
maximum = meli_api.get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 2313 resultados para nuestra consulta


In [19]:
item_list = meli_api.create_item_list(filters = filters)

In [20]:
data4 = meli_api.get_df_list(item_list)
data4["installment"] = "no_interest"
data4["display_size"] = "entre 14.1 y 16.9"
data4["shipping_cost"] = "free"

________________________

In [21]:
data = pd.concat([data, data2, data3, data4])

NameError: name 'data2' is not defined

In [ ]:
data.shape

(11655, 76)

In [ ]:
data.title.value_counts()

Renovada) Dell Inspiron 5482 2-in-1 14 Inch Touchscreen Fhd®    13
Renovada) Dell Inspiron 5482 2-in-1 14 Inch Touchscreen Fhd     13
Notebook Asus R410ma Intel Celeron 4gb 128gb Emmc Windows 10     9
Notebook Pcbox Fire 3 I3-1005g1 8gb Ram  Ssd 256 14  Win10       8
Renovada) Dell Latitude 7480 14 Touchscreen Laptop Intel Cor     7
                                                                ..
Notebook Lenovo I3 1005g1 8gb 128gb Ssd 14 Fhd W11s 81wd010u     1
Notebook Lenovo I3 1005g1 4gb 128gb Ssd 14 Fhd W11s 81wd010u     1
Notebook Acer I5 1135g7 8gb 256gb 14  Fhd W11 Nx.a25aa.002       1
Renovada) Microsoft Surface Laptop 2 Intel Core I7 8gb Ram 2     1
Cx 23800w                                                        1
Name: title, Length: 8881, dtype: int64

In [ ]:
import datetime as dt

anio = dt.datetime.now().year
mes = dt.datetime.now().month
dia = dt.datetime.now().day
hora = dt.datetime.now().hour
minuto = dt.datetime.now().minute


In [ ]:
path_save /f"datos_laptops_{anio*10000+mes*100+dia}_{hora}.{minuto}.csv"

WindowsPath('c:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis/data/datos_laptops_20220827_11.41.csv')

In [ ]:
data.to_csv(path_save / f"datos_laptops_{anio*10000+mes*100+dia}_{hora}.{minuto}.csv", index = False, sep = ";")